### 문장사이의 거리
- 평면이든, 공간이든 두 점 사이의 거리는 구할 수 있다.
- 만약 비교해야할 문장을 벡터로 표현할 수 있다면 거리를 구할 수 있다. 

In [1]:
#CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(min_df=1)

In [2]:
#훈련용 문장
contents=['상처받은 아이들은 너무 일찍 커버려',
         '내가 상처받은 거 아는 사람 불편해',
         '잘 사는 사람들은 좋은 사람 되기 쉬워',
         '아무 일도 아니야 괜찮아']

In [3]:
#Okt(이전 Twitter) 사용
from konlpy.tag import Okt
t=Okt()

In [4]:
#형태소 분석
contents_tokens=[t.morphs(row) for row in contents]
contents_tokens

[['상처', '받은', '아이', '들', '은', '너무', '일찍', '커버', '려'],
 ['내', '가', '상처', '받은', '거', '아는', '사람', '불편해'],
 ['잘', '사는', '사람', '들', '은', '좋은', '사람', '되기', '쉬워'],
 ['아무', '일도', '아니야', '괜찮아']]

In [5]:
#형태소 분석된 결과를 다시 문장으로 합친다.
#이 함수는 문장이 여러개일 경우에는 상당히 오래 걸릴 수 있으니 주의해라.
contents_for_vectorize=[]
for content in contents_tokens:
    sentence=''
    for word in content:
        sentence=sentence+' '+word
        
    contents_for_vectorize.append(sentence)
    
contents_for_vectorize

[' 상처 받은 아이 들 은 너무 일찍 커버 려',
 ' 내 가 상처 받은 거 아는 사람 불편해',
 ' 잘 사는 사람 들 은 좋은 사람 되기 쉬워',
 ' 아무 일도 아니야 괜찮아']

In [6]:
#Vectorize
X=vectorizer.fit_transform(contents_for_vectorize)
X

<4x17 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [7]:
#4개의 문장, 17개의 말뭉치
num_samples,num_features=X.shape
num_samples,num_features

(4, 17)

In [8]:
vectorizer.get_feature_names()

['괜찮아',
 '너무',
 '되기',
 '받은',
 '불편해',
 '사는',
 '사람',
 '상처',
 '쉬워',
 '아는',
 '아니야',
 '아무',
 '아이',
 '일도',
 '일찍',
 '좋은',
 '커버']

In [9]:
X.toarray().transpose()

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 2, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0]], dtype=int64)

In [10]:
#테스트용 문장
new_post=['상처받기 싫어 괜찮아']
new_post_tokens=[t.morphs(row) for row in new_post]
new_post_for_vectorize=[]

for content in new_post_tokens:
    sentence=''
    for word in content:
        sentence=sentence+' '+word
    new_post_for_vectorize.append(sentence)
    
new_post_for_vectorize

[' 상처 받기 싫어 괜찮아']

In [11]:
#벡터로 표현
new_post_vec =vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [12]:
#유클리드 거리
import scipy as sp

def dist_raw(v1,v2):
    delta = v1-v2
    return sp.linalg.norm(delta.toarray())

In [13]:
#결과
dist=[dist_raw(each,new_post_vec) for each in X]
dist

[2.449489742783178, 2.23606797749979, 3.1622776601683795, 2.0]

In [14]:
print('Best post is ',dist.index(min(dist)), ", dist =", min(dist))
print('test post is --> ', new_post)
print('best dist post is -->', contents[dist.index(min(dist))])

Best post is  3 , dist = 2.0
test post is -->  ['상처받기 싫어 괜찮아']
best dist post is --> 아무 일도 아니야 괜찮아


In [15]:
#유사도 -벡터사이의 거리
for i in range(0,len(contents)):
    print(X.getrow(i).toarray())
    
    print('---------------------')
    print(new_post_vec.toarray())

[[0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 1]]
---------------------
[[1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]]
[[0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0]]
---------------------
[[1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]]
[[0 0 1 0 0 1 2 0 1 0 0 0 0 0 0 1 0]]
---------------------
[[1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]]
[[1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0]]
---------------------
[[1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


### TF-IDF
- 한 문서에서 많이 등장한 단어에 가중치를 준다
- 전체 문서에서 많이 나타나는 단어는 가중치를 줄인다.

In [16]:
#TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(min_df=1, decode_error='ignore')

In [17]:
X=vectorizer.fit_transform(contents_for_vectorize)
num_samples, num_features= X.shape
num_samples, num_features

(4, 17)

In [18]:
X.toarray().transpose()

array([[0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.34431452, 0.40104275, 0.        , 0.        ],
       [0.        , 0.50867187, 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.        , 0.40104275, 0.6191303 , 0.        ],
       [0.34431452, 0.40104275, 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.        , 0.50867187, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.43671931, 0.        , 0.        , 0.        ]])

In [19]:
#테스트 문장 적용
new_post_vec=vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0.78528828, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.6191303 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [21]:
#거리를 구하는 또다른 방법 : normalize
def dist_norm(v1,v2):
    v1_normalized=v1/sp.linalg.norm(v1.toarray())
    v2_normalized=v2/sp.linalg.norm(v2.toarray())
    
    delta = v1_normalized- v2_normalized
    
    return sp.linalg.norm(delta.toarray())

In [22]:
#결과
dist=[dist_norm(each, new_post_vec) for each in X]

print('Best post is ', dist.index(min(dist)), ", dist = ", min(dist))
print('test post is --> ', new_post )
print('best dist post is --> ', contents[dist.index(min(dist))])

Best post is  3 , dist =  1.1021396119773588
test post is -->  ['상처받기 싫어 괜찮아']
best dist post is -->  아무 일도 아니야 괜찮아


### Naver Api 에 적용하기

In [23]:
#gen_search_url
import urllib.request
def gen_search_url(api_node, search_text, start_num, disp_num):
    base='https://openapi.naver.com/v1/search'
    node='/'+api_node+'.json'
    param_query='?query='+urllib.parse.quote(search_text)
    param_start='&start='+str(start_num)
    param_disp='&display='+str(disp_num)
    
    return base+node+param_query+param_start+param_disp

In [24]:
#get_result_onpage
import json
import datetime

def get_result_onpage(url):
    request=urllib.request.Request(url)
    request.add_header('X-Naver-Client-Id',client_id)
    request.add_header('X-Naver-Client-Secret',client_secret)
    
    response=urllib.request.urlopen(request)
    
    print('[{}] Url Request Success'.format(datetime.datetime.now()))
    
    return json.loads(response.read().decode('utf-8'))

In [25]:
#문장을 수집하자
client_id='client_id'
client_secret='client_secret'

url=gen_search_url('kin','파이썬',10,10)
one_result=get_result_onpage(url)
one_result

[2020-04-17 17:52:59.915547] Url Request Success


{'lastBuildDate': 'Fri, 17 Apr 2020 17:52:59 +0900',
 'total': 128695,
 'start': 10,
 'display': 10,
 'items': [{'title': '<b>파이썬</b> 공부 대충 얼마나 걸릴까요? (50)',
   'link': 'https://kin.naver.com/qna/detail.nhn?d1id=1&dirId=10402&docId=352398003&qb=7YyM7J207I2s&enc=utf8&section=kin.qna&rank=10&search_sort=0&spq=0',
   'description': '... 현재 학원에서 <b>파이썬</b> 반복문을 갓 입문했습니다. <b>파이썬</b> 말고 리눅스를 하고싶은데 <b>파이썬</b>은 보통 얼마나 하면... 컴퓨터,프로그래밍,IT 분야 지식IN 전문 답변인 진로진학취업상담가 이에요 <b>파이썬</b>이랑 리눅스 서버도... '},
  {'title': '<b>파이썬</b>프로그래밍을 배우면 일본취업에 유리할까요?',
   'link': 'https://kin.naver.com/qna/detail.nhn?d1id=1&dirId=10402&docId=349484451&qb=7YyM7J207I2s&enc=utf8&section=kin.qna&rank=11&search_sort=0&spq=0',
   'description': '궁금한게 있는데요 <b>파이썬</b>프로그래밍을 배우면 일본취업에 유리할까요? 일본은 대부분 c언어나 자바아닌가요? <b>파이썬</b>프로그래밍이 쓸모없지않겠죠? 사실 <b>파이썬</b>은.. 언어... 그 이유는 <b>파이썬</b>의 경우 보통 처음부터 뭔가 협업하여... '},
  {'title': '독학으로 <b>파이썬</b>',
   'link': 'https://kin.naver.com/qna/detail.nhn?d1id=1&dirId=10402&docId=351178975&qb=7YyM7J207I2s

In [26]:
one_result['items'][0]['description']

'... 현재 학원에서 <b>파이썬</b> 반복문을 갓 입문했습니다. <b>파이썬</b> 말고 리눅스를 하고싶은데 <b>파이썬</b>은 보통 얼마나 하면... 컴퓨터,프로그래밍,IT 분야 지식IN 전문 답변인 진로진학취업상담가 이에요 <b>파이썬</b>이랑 리눅스 서버도... '

In [27]:
#태그제거기
# d tag를 삭제하기
def delete_tag(input_str):
    input_str=input_str.replace('<b>',"")
    input_str=input_str.replace('</b>',"")
    
    return input_str

In [28]:
def get_description(pages):
    contents=[]
    for sentences in pages['items']:
        contents.append(delete_tag(sentences['description']))
    return contents

In [29]:
contents=get_description(one_result)
contents

['... 현재 학원에서 파이썬 반복문을 갓 입문했습니다. 파이썬 말고 리눅스를 하고싶은데 파이썬은 보통 얼마나 하면... 컴퓨터,프로그래밍,IT 분야 지식IN 전문 답변인 진로진학취업상담가 이에요 파이썬이랑 리눅스 서버도... ',
 '궁금한게 있는데요 파이썬프로그래밍을 배우면 일본취업에 유리할까요? 일본은 대부분 c언어나 자바아닌가요? 파이썬프로그래밍이 쓸모없지않겠죠? 사실 파이썬은.. 언어... 그 이유는 파이썬의 경우 보통 처음부터 뭔가 협업하여... ',
 '독학으로 파이썬을 공부해 보려고하는데요 인강과 교재? 책 추천 좀 해주세요.... 학원에서 파이썬 하나만 배우면 비용이 대체로 얼마정도 드나요?... 혼자서 파이썬을 독학하시기에는 어려운 점이 있으실 것이라는 생각이 듭니다. 많은... ',
 '파이썬 공부할 필요가 생겨서 파이썬 공부를 할까하는데 아무것도 모르는 비전공자라 파이썬은 어디에 쓰이고... 파이썬공부에 대해 질문 남겨주셨는데요, 우선, 파이썬은 홈페이지 개발에 많이 쓰입니다 윈도우 실행 프로그램을... ',
 '... 일단 파이썬으로 독학 시작해보려고하는데요 뭐 아는 게 없어서 선뜻 시작하기가 애매하네요 파이썬 독학으로 배우기 많이 어렵나요?? 요즘은 시대가 좋아서 파이썬 관련 서적이나 인강이 잘 돼있답니다 게다가 파이썬은 비교적... ',
 '파이썬언어 공부먼저 ?해야하나요 안녕하세요. IT... 먼저 파이썬에 대해서 궁금하신 이유는 무엇인가요? 파이썬은 프로그래밍언어라는 겁니다. 가장 인기 있는 언어죠. 컴퓨터 공학부나 , IT 기업에서도 파이썬을 가장 많이... ',
 '... 특성화고 다니는데 학교 수업 많으로는 파이썬, c언어... 부분이고 파이썬, c언어가 제대로 이해 못한다면 C++, 자바까지 배울 수 없어 파이썬, C언어가 가장 중요하다고 할 수 있습니다. 대부분의 파이썬인강, 인터넷강의는 옛날... ',
 '파이썬 가르쳐주는 학원이 어디죠? 파이썬 배울 수 있는 학원 추천해드릴께요! http://sbsdaejeon.co

In [30]:
#형태소 분석
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt

t=Okt()
vectorizer=CountVectorizer(min_df=1)

In [31]:
contents_tokens=[t.morphs(row)for row in contents]
contents_tokens

[['...',
  '현재',
  '학원',
  '에서',
  '파이썬',
  '반복',
  '문',
  '을',
  '갓',
  '입문',
  '했습니다',
  '.',
  '파이썬',
  '말고',
  '리눅스',
  '를',
  '하고싶은데',
  '파이썬',
  '은',
  '보통',
  '얼마나',
  '하면',
  '...',
  '컴퓨터',
  ',',
  '프로그래밍',
  ',',
  'IT',
  '분야',
  '지식',
  'IN',
  '전문',
  '답변',
  '인',
  '진로',
  '진학',
  '취업',
  '상담',
  '가',
  '이에요',
  '파이썬',
  '이랑',
  '리눅스',
  '서버',
  '도',
  '...'],
 ['궁금한게',
  '있는데요',
  '파이썬',
  '프로그래밍',
  '을',
  '배우면',
  '일',
  '본',
  '취업',
  '에',
  '유리할까',
  '요',
  '?',
  '일본',
  '은',
  '대부분',
  'c',
  '언어',
  '나',
  '자바',
  '아닌가요',
  '?',
  '파이썬',
  '프로그래밍',
  '이',
  '쓸모없지',
  '않겠죠',
  '?',
  '사실',
  '파이썬',
  '은',
  '..',
  '언어',
  '...',
  '그',
  '이유',
  '는',
  '파이썬',
  '의',
  '경우',
  '보통',
  '처음',
  '부터',
  '뭔가',
  '협업',
  '하여',
  '...'],
 ['독학',
  '으로',
  '파이썬',
  '을',
  '공부',
  '해',
  '보려고하는데요',
  '인강',
  '과',
  '교재',
  '?',
  '책',
  '추천',
  '좀',
  '해주세요',
  '....',
  '학원',
  '에서',
  '파이썬',
  '하나',
  '만',
  '배우면',
  '비용',
  '이',
  '대체로',
  '얼마',
  '정도',
  '드나',
  '요',


In [32]:
#vectorize후 다시 합치기
contents_for_vectorize=[]
for content in contents_tokens:
    sentence=''
    for word in content:
        sentence=sentence+' '+word
    contents_for_vectorize.append(sentence)
    
contents_for_vectorize

[' ... 현재 학원 에서 파이썬 반복 문 을 갓 입문 했습니다 . 파이썬 말고 리눅스 를 하고싶은데 파이썬 은 보통 얼마나 하면 ... 컴퓨터 , 프로그래밍 , IT 분야 지식 IN 전문 답변 인 진로 진학 취업 상담 가 이에요 파이썬 이랑 리눅스 서버 도 ...',
 ' 궁금한게 있는데요 파이썬 프로그래밍 을 배우면 일 본 취업 에 유리할까 요 ? 일본 은 대부분 c 언어 나 자바 아닌가요 ? 파이썬 프로그래밍 이 쓸모없지 않겠죠 ? 사실 파이썬 은 .. 언어 ... 그 이유 는 파이썬 의 경우 보통 처음 부터 뭔가 협업 하여 ...',
 ' 독학 으로 파이썬 을 공부 해 보려고하는데요 인강 과 교재 ? 책 추천 좀 해주세요 .... 학원 에서 파이썬 하나 만 배우면 비용 이 대체로 얼마 정도 드나 요 ?... 혼자 서 파이썬 을 독학 하 시기 에는 어려운 점 이 있으실 것 이라는 생각 이 듭니 다 . 많은 ...',
 ' 파이썬 공부 할 필요 가 생겨서 파이썬 공부 를 할까 하는데 아무 것 도 모르는 비 전공자 라 파이썬 은 어디 에 쓰이고 ... 파이썬 공부 에 대해 질문 남겨주셨는데요 , 우선 , 파이썬 은 홈페이지 개발 에 많이 쓰입니다 윈도우 실행 프로그램 을 ...',
 ' ... 일단 파이썬 으로 독학 시작 해보려고하는데요 뭐 아는 게 없어서 선뜻 시작 하기가 애매하네요 파이썬 독학 으로 배우기 많이 어렵나요 ?? 요즘 은 시대 가 좋아서 파이썬 관련 서적 이나 인강 이 잘 돼있답니다 게다가 파이썬 은 비교 적 ...',
 ' 파이썬 언어 공부 먼저 ? 해야하나요 안녕하세요 . IT ... 먼저 파이썬 에 대해 서 궁금하신 이유 는 무엇 인가요 ? 파이썬 은 프로그래밍언어 라는 겁니다 . 가장 인기 있는 언어 죠 . 컴퓨터 공 학부 나 , IT 기업 에서도 파이썬 을 가장 많이 ...',
 ' ... 특성화고 다니는데 학교 수업 많으로는 파이썬 , c 언어 ... 부분 이고 파이썬 , c 언어 가 제대로 이해 못 한다면 C ++, 자바 까지 

In [33]:
#결과
X=vectorizer.fit_transform(contents_for_vectorize)
X

<10x200 sparse matrix of type '<class 'numpy.int64'>'
	with 245 stored elements in Compressed Sparse Row format>

In [34]:
num_samples,num_features=X.shape
num_samples,num_features

(10, 200)

In [35]:
#vectorize 결과
vectorizer.get_feature_names()

['co',
 'http',
 'in',
 'it',
 'kr',
 'next',
 'previous',
 'pused',
 'sbs',
 'sbsdaejeon',
 '가능하다고',
 '가르쳐주는',
 '가장',
 '강의',
 '개발',
 '겁니다',
 '게다가',
 '경우',
 '공부',
 '과정',
 '관련',
 '관심',
 '교육',
 '교육과정',
 '교재',
 '구축',
 '궁금하신',
 '궁금한게',
 '그런',
 '기술',
 '기업',
 '까지',
 '깨끗한',
 '남겨주셨는데요',
 '다녀',
 '다니는데',
 '다양하고',
 '답변',
 '대부분',
 '대체로',
 '대해',
 '독학',
 '돼있답니다',
 '두면',
 '드나',
 '들은',
 '듭니',
 '라는',
 '리눅스',
 '많으로는',
 '많은',
 '많이',
 '말고',
 '먼저',
 '모르는',
 '무엇',
 '뭔가',
 '박세진',
 '반복',
 '배우기',
 '배우는거라',
 '배우면',
 '배울',
 '배워',
 '보려고',
 '보려고하는데요',
 '보통',
 '부분',
 '부터',
 '분야',
 '불러서',
 '비교',
 '비용',
 '사람',
 '사실',
 '사업가',
 '상담',
 '생각',
 '생겨서',
 '생긴',
 '서버',
 '서적',
 '선뜻',
 '수강',
 '수업',
 '시기',
 '시대',
 '시작',
 '실습',
 '실행',
 '쓰이고',
 '쓰입니다',
 '쓸모없지',
 '아는',
 '아닌가요',
 '아무',
 '아무래도',
 '아카데미',
 '안녕하세요',
 '않겠죠',
 '알려주는',
 '알아보는거',
 '애매하네요',
 '어느',
 '어디',
 '어려운',
 '어렵나요',
 '언어',
 '얼마',
 '얼마나',
 '없어',
 '없어서',
 '없을까요',
 '에는',
 '에서',
 '에서도',
 '예요',
 '옛날',
 '요즘',
 '우선',
 '웹페이지',
 '위치',
 '윈도우',
 '유리할까',
 '으로',
 '의뢰',
 '이고',
 '이나'

In [36]:
#유사도 측정할 테스트 문장
new_post=['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
new_post_tokens=[t.morphs(row) for row in new_post]

new_post_for_vectorize=[]
for content in new_post_tokens:
    sentence=''
    for word in content:
        sentence=sentence+ ' '+ word
        
    new_post_for_vectorize.append(sentence)
    
new_post_for_vectorize

[' 파이썬 을 배우는데 좋은 방법 이 어떤 것 인지 추천 해주세요']

In [37]:
#vectorize
new_post_vec= vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0]], dtype=int64)

In [38]:
#유클리드 거리
import scipy as sp
def dist_raw(v1,v2):
    delta=v1-v2
    return sp.linalg.norm(delta.toarray())

In [39]:
dist=[dist_raw(each, new_post_vec)for each in X]
dist

[6.4031242374328485,
 6.244997998398398,
 5.5677643628300215,
 6.855654600401044,
 6.557438524302,
 6.708203932499369,
 6.48074069840786,
 6.4031242374328485,
 6.6332495807108,
 6.928203230275509]

In [40]:
#결과보기
print('Best post is', dist.index(min(dist)), ", dist = ", min(dist))
print('test post is --> ', new_post)
print('best dist post is --> ', contents[dist.index(min(dist))])

Best post is 2 , dist =  5.5677643628300215
test post is -->  ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
best dist post is -->  독학으로 파이썬을 공부해 보려고하는데요 인강과 교재? 책 추천 좀 해주세요.... 학원에서 파이썬 하나만 배우면 비용이 대체로 얼마정도 드나요?... 혼자서 파이썬을 독학하시기에는 어려운 점이 있으실 것이라는 생각이 듭니다. 많은... 


In [41]:
#거리를 구하는 또다른 방법 : normalize
def dist_norm(v1,v2):
    v1_normalized=v1/sp.linalg.norm(v1.toarray())
    v2_normalized=v2/sp.linalg.norm(v2.toarray())
    
    delta = v1_normalized- v2_normalized
    
    return sp.linalg.norm(delta.toarray())

In [42]:
dist=[dist_norm(each, new_post_vec) for each in X]

print('Best post is ', dist.index(min(dist)), ", dist = ", min(dist))
print('test post is --> ', new_post )
print('best dist post is --> ', contents[dist.index(min(dist))])

Best post is  2 , dist =  1.0312197575775526
test post is -->  ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
best dist post is -->  독학으로 파이썬을 공부해 보려고하는데요 인강과 교재? 책 추천 좀 해주세요.... 학원에서 파이썬 하나만 배우면 비용이 대체로 얼마정도 드나요?... 혼자서 파이썬을 독학하시기에는 어려운 점이 있으실 것이라는 생각이 듭니다. 많은... 


In [43]:
#Tfidf
def tfidf(t,d,D):
    tf=float(d.count(t))/sum(d.count(w) for w in set(d))
    idf=sp.log(float(len(D)/(len([doc for doc in D if t in doc]))))
    return tf * idf

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(min_df=1, decode_error='ignore')

In [45]:
X=vectorizer.fit_transform(contents_for_vectorize)
num_samples, num_features = X.shape
num_samples, num_features

(10, 200)

In [46]:
new_post_vec=vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [47]:
#결과
dist=[dist_norm(each, new_post_vec) for each in X]

print('Best post is ', dist.index(min(dist)), ", dist = ", min(dist))
print('test post is --> ', new_post )
print('best dist post is --> ', contents[dist.index(min(dist))])

Best post is  2 , dist =  1.1788852025234768
test post is -->  ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
best dist post is -->  독학으로 파이썬을 공부해 보려고하는데요 인강과 교재? 책 추천 좀 해주세요.... 학원에서 파이썬 하나만 배우면 비용이 대체로 얼마정도 드나요?... 혼자서 파이썬을 독학하시기에는 어려운 점이 있으실 것이라는 생각이 듭니다. 많은... 
